In [ ]:
# Sample data for buyers and sellers
buyer1 = {'agent_id': 41544, 'Year': 2022}
buyer2 = {'agent_id': 41545, 'Year': 2022}
seller1 = {'agent_id': 41546, 'ProductionId': 'P1'}
seller2 = {'agent_id': 41547, 'ProductionId': 'P2'}

# Sample transactions
transactions = [
    {'buyer': buyer1, 'seller': seller1, 'transaction_price': 10000, 'land_exchange_perc': 0.2, 'land_exchange': 10},
    {'buyer': buyer2, 'seller': seller2, 'transaction_price': 15000, 'land_exchange_perc': 0.3, 'land_exchange': 15},
]

# Initialize the list for matched transactions
matched_transactions = []

# Append transactions to the list
for transaction in transactions:
    buyer = transaction['buyer']
    seller = transaction['seller']
    transaction_price = transaction['transaction_price']
    land_exchange_perc = transaction['land_exchange_perc']
    land_exchange = transaction['land_exchange']

    matched_transactions.append({
        'ProductionId': seller['ProductionId'],
        'YearID': buyer['Year'],
        'Buyer_FarmId': buyer['agent_id'],
        'Seller_FarmId': seller['agent_id'],
        'SalePrice': transaction_price,
        'Percentage': land_exchange_perc,
        'Land_exchange': land_exchange
    })

# Print the final matched_transactions list
print(matched_transactions)


In [ ]:
# Assume you want to find transactions related to agent_id = 2
desired_agent_id = 10
# matched_transactions = []
# Iterate through matched_transactions to find transactions with the desired_agent_id
matching_transactions = [transaction for transaction in matched_transactions if transaction['Buyer_FarmId'] == desired_agent_id or transaction['Seller_FarmId'] == desired_agent_id]

# Print the matching transactions
print(matching_transactions)


In [ ]:
# Example: Extract SalePrice for agent_id = 2
desired_agent_id = 2

# Find matching transactions
matching_transactions = [transaction for transaction in matched_transactions if transaction['Buyer_FarmId'] == desired_agent_id or transaction['Seller_FarmId'] == desired_agent_id]

# Initialize a variable to store the total SalePrice
total_sale_price = 0


# Iterate through matching transactions and accumulate SalePrice
for transaction in matching_transactions:
    total_sale_price += transaction['SalePrice']

# Print the total SalePrice
print(f"Total SalePrice for agent_id {desired_agent_id}: {total_sale_price}")

if transaction['Buyer_FarmId'] == desired_agent_id:
    buyer_role = True
if transaction['Seller_FarmId'] == desired_agent_id:
    seller_role = True

In [2]:
import pandas as pd
import numpy as np
# LP_LMM_data = pd.read_csv("agents_LP_LMM.csv", index_col=0)
# productions_LMM = pd.read_csv("productions_LMM.csv", index_col=0)

In [3]:
def buy_sell_match(buyers_c, sellers_c):
    min_u1_diff = 100000

    for _, buyer in buyers_c.iterrows():
        for _, seller in sellers_c.iterrows():
            u1_diff = np.abs(buyer["u1_f"] + seller["u1_f"])     
            
            if u1_diff < min_u1_diff:
                min_u1_diff = u1_diff
                seller_id = int(seller["agent_id"])
                buyer_id = int(buyer["agent_id"])
    return seller_id, buyer_id

        

In [ ]:
seller_productionId = []
matched_transactions = []

buyers = LP_LMM_data[LP_LMM_data['u1_f'] > 0]
sellers = LP_LMM_data[LP_LMM_data['u1_f'] < 0]    

buyers_c = buyers.copy()
sellers_c = sellers.copy()

while len(buyers_c) > 0 and len(sellers_c) > 0:
    # Match between sellers and buyers
    seller_id, buyer_id = buy_sell_match(buyers_c, sellers_c)

    # Extract agents
    buyer = buyers[buyers["agent_id"] == buyer_id]
    seller = sellers[sellers["agent_id"] == seller_id]

    buyer_productions = productions_LMM[productions_LMM["agent_id"] == buyer_id]
    seller_productions = productions_LMM[productions_LMM["agent_id"] == seller_id]

    buyer_offer = buyer['u1_f'].values[0]
    seller_offer = -seller['u1_f'].values[0]

    buyer_offer_ha = buyer['u1_ha_f'].values[0]
    seller_offer_ha = -seller['u1_ha_f'].values[0]
    # Perform transaction 
    # 1- Set the sale price and land exchanged
    if buyer_offer < seller_offer:
        transaction_price = buyer_offer
        if buyer_offer_ha < seller_offer_ha:
            land_exchange = (buyer_offer * seller_offer_ha) / (seller_offer)
        else:
            land_exchange = - seller['u1_ha_f'].values[0]
    else: 
        transaction_price = seller_offer
        land_exchange = seller_offer_ha

    # 2.1 - Check if the seller got several productions
    if len(seller_productions) > 1:
        land_exchange_c = land_exchange
        seller_productions_sorted = seller_productions.sort_values("land_ha", ascending = False)

        for _, seller_prod in seller_productions_sorted.iterrows():
            if seller_prod["land_ha"] >= land_exchange_c:
                seller_productionId.append(seller_prod["ProductionId"])
                land_transfer_perc = (land_exchange / seller_productions['land_ha'].values[0])
                break
            else:
                land_exchange_c = land_exchange_c - seller_prod["land_ha"]
                seller_productionId.append(seller_prod["ProductionId"])
                land_transfer_perc = 1.0
                if land_exchange_c == 0:
                    break

    else:
        seller_productionId.append(seller_productions["ProductionId"].values[0])
        land_transfer_perc = (land_exchange / seller_productions['land_ha'].values[0])



    # Drop agent
    sellers_c = sellers_c[sellers_c['agent_id'] != seller_id]
    buyers_c = buyers_c[buyers_c['agent_id'] != buyer_id]

    # Record the transaction -- TODO-3: Store all the values in the same dataset, improve efficiency
    for prod_Id in seller_productionId:
        matched_transactions.append({
            'ProductionId': prod_Id,
            'YearID': seller_productions['year_id'].values[0],
            'Year': seller['Year'].values[0],
            'Buyer_FarmId': buyer_id,
            'Seller_FarmId': seller_id,
            'SalePrice': transaction_price,
            'Percentage': land_transfer_perc,
            'Land_exchange': land_exchange
        })



In [ ]:
matched_transactions

In [7]:
def gen_transactions_samples():
    # Sample data for buyers and sellers
    buyer1 = {'agent_id': 41544, 'Year': 2022}
    buyer2 = {'agent_id': 41545, 'Year': 2022}
    seller1 = {'agent_id': 41546, 'ProductionId': 'P1'}
    seller2 = {'agent_id': 41547, 'ProductionId': 'P2'}

    # Sample transactions
    transactions = [
        {'buyer': buyer1, 'seller': seller1, 'transaction_price': 10000, 'land_exchange_perc': 0.2, 'land_exchange': 10},
        {'buyer': buyer2, 'seller': seller2, 'transaction_price': 15000, 'land_exchange_perc': 0.3, 'land_exchange': 15},
    ]

    # Initialize the list for matched transactions
    matched_transactions = []

    # Append transactions to the list
    for transaction in transactions:
        buyer = transaction['buyer']
        seller = transaction['seller']
        transaction_price = transaction['transaction_price']
        land_exchange_perc = transaction['land_exchange_perc']
        land_exchange = transaction['land_exchange']

        matched_transactions.append({
            'ProductionId': seller['ProductionId'],
            'YearID': buyer['Year'],
            'Buyer_FarmId': buyer['agent_id'],
            'Seller_FarmId': seller['agent_id'],
            'SalePrice': transaction_price,
            'Percentage': land_exchange_perc,
            'Land_exchange': land_exchange
        })
    return matched_transactions

In [ ]:
def LMM(agents_LMM_data):
    # Separate buyers and sellers
    buyers = agents_LMM_data[agents_LMM_data['u1_f'] > 0]
    sellers = agents_LMM_data[agents_LMM_data['u1_f'] < 0]    

    # Match buyers and sellers
    matched_transactions = []

    # Check if there are buyers and sellers
    if buyers.shape[0] != 0 and sellers.shape[0] != 0:
        buyers_sorted = buyers.sort_values(by = ["u1_f", "u1_ha_f"])
        sellers_sorted = sellers.sort_values(by = ["u1_f", "u1_ha_f"])
        

        for _, buyer in buyers.iterrows():
            # TODO-1: Iterate through ProductionID - consider AgentID with several ProductionID linked. 
            # Sort sellers based on the specified objectives
            sorted_auctions = sellers.copy()
            sorted_auctions['action_diff'] = np.abs(sorted_sellers['u1_f'] + buyer['u1_f'])
            sorted_auctions['u0_ha_diff'] = np.abs(sorted_sellers['u1_ha_f'] - buyer['u1_ha_f'])
            sorted_auctions['land_value_diff'] = np.abs(buyer['a_va_f'] - (sorted_sellers['land_value']))

            sorted_sellers = sorted_sellers.sort_values(by=['action_diff', 'land_value_diff', 'u0_ha_diff'])

            for _, seller in sorted_sellers.iterrows():
                seller_productions = agents_LMM_data[agents_LMM_data['agent_id'] == seller['agent_id']]
                seller_productions = seller_productions.sort_values(by='u1_f', ascending=True)

                # Find match and calculate the transaction 
                transaction_price = min(buyer['u1_f'], - seller['u1_f'])
                land_exchange = transaction_price * seller['land_value']

                land_exchange_perc = 100*(land_exchange / seller['land_ha'])

                if land_exchange_perc > 100:
                #Check if the land exchanged is above the total land associated with the ProductionId
                # TODO-2: Once completed TODO-1 ensure that the land exchange is not above the land linked to the ProducitonID
                # *Consider selling different production IDs
                    land_exchange = transaction_price * seller['land_ha']
                    land_exchange_perc = 100


                # Record the transaction -- TODO-3: Store all the values in the same dataset, improve efficiency
                matched_transactions_LP.append({
                    'ProductionId': seller['ProductionId'],
                    'YearID': buyer['Year'],
                    'Buyer_FarmId': buyer['agent_id'],
                    'Seller_FarmId': seller['agent_id'],
                    'SalePrice': transaction_price,
                    'Percentage': land_exchange_perc,
                    'Land_exchange': land_exchange
                })

                matched_transactions.append({
                    'ProductionId': seller['ProductionId'],
                    'YearID': buyer['Year'],
                    'FarmId': buyer['agent_id'],
                    'SalePrice': transaction_price,
                    'Percentage': land_exchange_perc,
                })                


                # Update buyer and seller budgets and land holdings
                buyer['u1_ha_f'] -= land_exchange
                seller['u1_ha_f'] += land_exchange

                buyer['u1_f'] -= transaction_price
                seller['u1_f'] += transaction_price

                break

    else: 
        print("No buyers or sellers")

    matched_transactions = pd.DataFrame(matched_transactions)
    matched_transactions_LP = pd.DataFrame(matched_transactions_LP)
    return matched_transactions_LP, matched_transactions 


In [ ]:
seller_production_sold:list[tuple:(int, float)] = []

seller_production_sold.append((23, 4))
seller_production_sold.append((20, 0.7))
seller_production_sold

## LP optimisation results evaluation

In [1]:
import pandas as pd

In [42]:
LP_res = pd.read_csv("agents_LP_optimisation_results.csv", index_col=0)

In [ ]:
LP_res

In [ ]:
filtered_df = LP_res[LP_res["lt_f"] < LP_res["lt_f"].quantile(0.80)]

# filtered_df[filtered_df["Year"] == 2020]["lt_f"].hist(bins = 50)
# filtered_df[filtered_df["Year"] == 2021]["lt_f"].hist(bins = 50)
# filtered_df[filtered_df["Year"] == 2022]["fni_f"].hist(bins = 50)
# filtered_df[filtered_df["Year"] == 2023]["fni_f"].hist(bins = 50)
# filtered_df[filtered_df["Year"] == 2024]["fni_f"].hist(bins = 50)

LP_res[LP_res["Year"] == 2021]["u1_f"].hist(bins = 50)
# LP_res[LP_res["Year"] == 2022]["u1_f"].hist(bins = 50)
# LP_res[LP_res["Year"] == 2023]["u1_f"].hist(bins = 50)
# LP_res[LP_res["Year"] == 2024]["fni_f"].hist(bins = 50)

In [49]:
holder_age_aux = range(24, 64)

EPS = 0.1 

In [ ]:
holder_age_aux 

In [ ]:

def SR_decay(holder_age, SR_ini=0.5, holder_age_min=30, decay_type='lineal'):
    
    years_range = 55 - holder_age_min
    
    if holder_age > 55:
        SR_obj = 0
        return SR_obj

    if decay_type == 'lineal':
        total_decay = SR_ini
        decay_per_year = total_decay / years_range
        SR_decay = (holder_age - holder_age_min) * decay_per_year
    
    elif decay_type == 'log':
        if holder_age == holder_age_min:
            SR_decay = 0
        else:
            decay_factor = math.log(holder_age - holder_age_min + 1) / math.log(years_range + 1)
            SR_decay = SR_ini * decay_factor

    # Calcular el nuevo solvency ratio
    SR_obj = SR_ini - SR_decay
    
    # Asegurarse de que el solvency ratio no sea negativo
    if SR_obj < 0:
        SR_obj = 0
    
    return SR_obj

In [ ]:
import os
import sys
import datetime as dt
import pandas as pd
import numpy as np
import gurobipy as gp   #Import library as gp in order to know which functions are from gurobi
from gurobipy import GRB
import matplotlib.pyplot as plt
import httpx
from pydantic import parse_obj_as
from fastapi.encoders import jsonable_encoder
from agricore_sp_models.simulation_models import *
from agricore_sp_models.agricore_sp_models import *
from agricore_sp_models.logging import configure_orm_logger
from agricore_sp_models.logging import configure_orm_logger
from typing import List
from collections import defaultdict
from loguru import logger
from contextlib import nullcontext
from loguru import logger
from contextlib import nullcontext


from settings import settings

"""
Financial optimization function to use the financial model designed to AGRICORE by Idener
"""

def test_production_data(populationId, yearNumber):
    try:
        url = f'{settings.ORM_ENDPOINT}/population/{populationId}/farms/get/simulationdata/longperiod'
        params = {'year': yearNumber}
        client = httpx.Client()
        response = client.get(url, params=params, timeout=None)
        if response.status_code == 200:
            response_json = response.json()
            response_object = parse_obj_as(DataToLPDTO, response_json)
            result = process_inputs(response_object, yearNumber)
    except Exception as e:
        print(e)
        return None


def process_inputs(input: DataToLPDTO, yearNumber:int, simulationRunId = 0) -> (AgroManagementDecisionFromLP):

    outputData:List[AgroManagementDecisions]=[]
    errors:List[int] = []
    land_Transactions = []

    if simulationRunId != 0:
        configure_orm_logger(settings.ORM_ENDPOINT)
    with logger.contextualize(simulationRunId=simulationRunId, logSource="long_term_worker") if simulationRunId != 0 else nullcontext():
        logger.info(f"Received request to solve LT for yearNumber: {yearNumber}. With LT_Ignore: {input.ignoreLP} and LMM_ignore: {input.ignoreLMM}")

        agents:List[ValueToLPDTO] = input.values
        productions:List[AgriculturalProductionDTO] = input.agriculturalProductions
        year:int = yearNumber

        LT_ignore:bool = input.ignoreLP
        LMM_ignore:bool = input.ignoreLMM   # Indicates if the LMM has to be executed or not - If True, skip to Phase 3 

        try: 
            if not LT_ignore:
                result_df = pd.DataFrame()

                # Calculate ha average value (€/ha) of all the agents
                ha_va = [prod.landValue/prod.cultivatedArea if prod.cultivatedArea != 0 else 0 for prod in productions ]
                average_ha_va = np.mean(ha_va)

                # Extract Agents total ha
                # Calculate Agent average production (€) and costs (€) for its productions --> Use later to calculate the GFI
                agents_ha = defaultdict(float)
                agents_prod = defaultdict(lambda: {'prod_profit': 0, 'prod_costs': 0, 'land_value': 0, 'cultivated_area': 0, 'prod_count': 0})
                for prod in productions:
                    farm_id = prod.farmId    
                    
                    agents_ha[farm_id] += prod.cultivatedArea
                    agents_prod[farm_id]['prod_profit'] += prod.valueSales 
                    # agents_prod[farm_id]['prod_profit'] += prod.cropProduction #TODO: Check if cropProduction is € or tons. If €, use this.
                    agents_prod[farm_id]['prod_costs'] += prod.variableCosts
                    agents_prod[farm_id]['land_value'] += prod.landValue
                    agents_prod[farm_id]['cultivated_area'] += prod.cultivatedArea  
                    agents_prod[farm_id]['prod_count'] += 1

                # Calculate the average profit, costs, land value and cultivated area per agent (avg if the agent has more than one production)
                # agents_avg_prod_values = {
                #     group: {
                #         'prod_profit_avg': data['prod_profit'] / data['prod_count'],
                #         'prod_costs_avg': data['prod_costs'] / data['prod_count'],
                #         'land_value_avg': data['land_value'] / data['prod_count'],
                #         'cultivated_area_avg': data['cultivated_area'] / data['prod_count']
                #     }
                #     for group, data in agents_prod.items()
                # }
                    
                agents_avg_prod_values = {}

                for group, data in agents_prod.items():

                    prod_per_land_ha = 0
                    costs_per_land_ha = 0 

                    prod_per_land_value = 0
                    costs_per_land_value = 0

                    if data['land_value'] != 0 and data['cultivated_area'] != 0:
                        prod_per_land_ha = (data['prod_profit'] / data['prod_count']) / (data['cultivated_area'] / data['prod_count'])
                        costs_per_land_ha = (data['prod_costs'] / data['prod_count']) / (data['cultivated_area'] / data['prod_count'])

                        prod_per_land_value = (data['prod_profit'] / data['prod_count']) / (data['land_value'] / data['prod_count'])
                        costs_per_land_value = (data['prod_costs'] / data['prod_count']) / (data['land_value'] / data['prod_count'])
                    
                    agents_avg_prod_values[group] = {
                        group: {
                            'prod_per_land_ha' : prod_per_land_ha,
                            'costs_per_land_ha' : costs_per_land_ha,
                            'prod_per_land_value' : prod_per_land_value,
                            'costs_per_land_value' : costs_per_land_value
                        }
                    }
                    
                # Calculate the average profit and costs per land value for each agent

                # Average values of profit and costs to set as default values for agents without land
                prod_per_land_sum = 0
                costs_per_land_sum = 0
                for group in agents_avg_prod_values.keys():
                    prod_per_land_sum += agents_avg_prod_values[group][group]["prod_per_land_value"]
                    costs_per_land_sum += agents_avg_prod_values[group][group]["costs_per_land_value"]

                avg_profit = prod_per_land_sum / len(agents_avg_prod_values)
                avg_costs = costs_per_land_sum / len(agents_avg_prod_values)

                # Set default values to the avg, in case there is an agent witout productions.
                agents_prod.default_factory = lambda: {'prod_per_land_value': avg_profit, 'costs_per_land_value': avg_costs, 'prod_count': 0}

                if LMM_ignore == False:
                    #---------- Phase 1 - Execute LP optmisation for all agents ----------#
                    for agent in agents:
                        
                        agent_total_ha = agents_ha[agent.farmId]
                        agent_avg_prod = agents_avg_prod_values[agent.farmId][agent.farmId]["prod_per_land_value"]
                        agent_avg_costs = agents_avg_prod_values[agent.farmId][agent.farmId]["costs_per_land_value"]

                        # Subsidies
                        subsidies_nc = 0
                        for agentSubsidies in agent.agentSubsidies:
                            subsidies_nc += agentSubsidies.value

                        result = algorithm(SE465=agent.sE465, A_0=agent.a_0, SE490=agent.sE490, SE420=agent.sE420, SE410=agent.sE410, agents_id=agent.farmId, land_avg_prod = agent_avg_prod, land_avg_costs = agent_avg_costs, NFM = agent.agentHolder.holderFamilyMembers, SBT_nc = subsidies_nc, average_ha_va = average_ha_va, AversionRisk=agent.aversionRiskFactor, land_ha = agent_total_ha, year = year)  
                        if "error" in result:
                            errors.append(agent.farmId)
                        else:
                            result_df = pd.concat([result_df, result])

                    # Save Phase 1 agents' results
                    agents_LP_optimisation = result_df[result_df["Year"] == year + 1]

                    #------------------- Phase 2 - Execute LMM -------------------#
                    print("First phase completed. Starting phase 2")
                    # Data preparation
                    # -- LP first run results extraction
                    agents_LP_LMM = agents_LP_optimisation[["agent_id","Year", "u1_f", "u1_ha_f"]]

                    # -- Productions extractions
                    agent_id = []
                    year_id = []
                    land_ha = []
                    land_value = []
                    productionId = []
                    cropProduction = []
                    for production in productions:
                        # Do not consider farms that failed the previous optimization
                        if production.farmId not in errors:
                            agent_id.append(production.farmId)
                            year_id.append(production.yearId)
                            land_ha.append(production.cultivatedArea)
                            productionId.append(production.id)
                            cropProduction.append(production.cropProduction)
                            land_value.append(production.landValue)

                    productions_LMM = pd.DataFrame()
                    productions_LMM['agent_id'] = agent_id
                    productions_LMM['year_id'] = year_id
                    productions_LMM['land_ha'] = land_ha
                    productions_LMM['land_value'] = land_value
                    productions_LMM['ProductionId'] = productionId
                    productions_LMM['cropProduction'] = cropProduction
                    productions_LMM["ha_value"] = productions_LMM["land_value"] / productions_LMM["land_ha"]

                    # Execute LMM
                    LMM_transactions, land_Transactions = LMM(agents_LP_LMM, productions_LMM)

                else:
                    LMM_transactions = [] 
                    land_Transactions = []

                # ------------------- Phase 3 - Execute LP optimisation with transactions information -------------------#
                
                #Simulate LMM output
                # transactions_data_LP = [] # No outputs
                # transactions = []
                # transactions_data_LP = gen_transactions_samples() # Output samples
                
                print("Second phase completed. Starting phase 3")
                outputData:[AgroManagementDecisions] = []
                errors:[int] = []

                for agent in agents:
                    # Do not consider farms that failed the previous optimization
                    if agent.farmId not in errors:
                        # agent_land_ha = productions.CultivatedArea
                        agent_total_ha = agents_ha[agent.farmId]
                        agent_avg_prod = agents_avg_prod_values[agent.farmId][agent.farmId]["prod_per_land_value"]
                        agent_avg_costs = agents_avg_prod_values[agent.farmId][agent.farmId]["costs_per_land_value"]

                        # Subsidies
                        subsidies_nc = 0
                        for agentSubsidies in agent.agentSubsidies:
                            subsidies_nc += agentSubsidies.value

                        result = algorithm(SE465=agent.sE465, A_0= agent.a_0, SE490=agent.sE490, SE420=agent.sE420, SE410=agent.sE410, agents_id=agent.farmId, land_avg_prod = agent_avg_prod, land_avg_costs = agent_avg_costs, NFM = agent.agentHolder.holderFamilyMembers, SBT_nc = subsidies_nc, average_ha_va = average_ha_va, year = year, land_ha = agent_total_ha, AversionRisk=agent.aversionRiskFactor, LP_second_phase = True, LMM_transactions = LMM_transactions)  
                        
                        if "error" in result:
                            errors.append(agent.farmId)
                        else:
                            result_dict = result.to_dict()

                            #Evaluate holder retirment
                            if agent.agentHolder.holderSuccessors != 0:
                                holder_retire = holder_retire_ev(agent.agentHolder.holderAge, agent.agentHolder.holderSuccessorsAge)   
                            else:
                                holder_retire = False
                            
                            # Save results
                            agent_result = AgroManagementDecisions (
                                farmId = agent.farmId,
                                yearId= agent.yearId, 
                                agriculturalLand= round(result_dict['a_f'][1],2),
                                longAndMediumTermLoans= round(result_dict['lt_f'][1],2),
                                totalCurrentAssets= round(result_dict['d_f'][1],2),
                                averageLandValue=result_dict['a_va_f'][1],
                                targetedLandAquisitionArea= round(result_dict['u1_ha_f'][1],2), 
                                targetedLandAquisitionHectarPrice= round(result_dict['a_va_f'][1],2),
                                retireAndHandOver = holder_retire              
                            )
                            outputData.append(agent_result)
            else:
                for agent in agents:
                    agent_result = AgroManagementDecisions (
                        farmId = agent.farmId,
                        yearId= 0,
                        averageHAPrice= agent.averageHAPrice,
                        agriculturalLand= agent.averageHAPrice/agent.a_0 if agent.a_0 != 0 else 0,
                        totalCurrentAssets= agent.sE465,
                        longAndMediumTermLoans= agent.sE490,
                        targetedLandAquisitionArea= 0, 
                        targetedLandAquisitionHectarPrice= 0,
                        retireAndHandOver= False,
                        averageLandValue=agent.a_0
                    )
                    outputData.append(agent_result)
                land_Transactions = []
        except Exception as err:
            logger.error(f"LT failed. Exception {err=}, {type(err)=}")
            raise err

    result = AgroManagementDecisionFromLP(
        agroManagementDecisions = outputData,
        landTransactions = [] if len(land_Transactions) == 0 else land_Transactions,
        errorList = errors
    )
    return result


def multi_objective_scalarization(model, objectives):
    # Create a new variable for the scalarized objective
    z = model.addVar(name="z")

    weights = np.ones(len(objectives))

    # Scalarization: z = w1 * Y[0] + w2 * Y[1] + ... + wn * Y[n-1]
    model.addConstr(z == gp.quicksum(weights[i] * objectives[i] for i in range(len(objectives))), name="scalarization")

    # Set the objective to minimize the scalarized variable
    # model.setObjective(z, GRB.MINIMIZE)
    model.setObjective(z, GRB.MAXIMIZE)

def list_check_input(input):
    if isinstance(input, list):
        pass
    else:
        input = [input]
    return input

def algorithm(SE465, A_0, SE490, SE420, SE410, agents_id, land_ha, land_avg_prod, land_avg_costs, AversionRisk = 0.9,  year = 2018,  VAT = 0, NFM = 4, EFT = 10000, SBT_c = 0, SBT_nc = 0, average_ha_va = 2500, sim_period = 7, LP_second_phase = False, LMM_transactions = None):
    """
    Function to optimize using the financial model designed

    -- Parameters - inputs
    ----------
    SE465 = Total current assets
    SE420 = Farm net income
    SE410 = Gross farm income
    SE490 = Long & Medium term loans
    A_0 = Agricultural land - value (€)
    agent_ids 
    A_ha = Agricultural land (ha) - B_UO_10_A
    
    AversionRiskFactor                              (default value = 0.5) 
    VAT = Value added tax                           (default value = 0) 
    NFM = Number of members of the farmer's family  (default value = 4)
    EFT = External factors - costs                  (default value = 10.000)
    SBT_c = coupled subsidies                       (default value = 0)         
    SBT_nc = not coupled subsidies                  (default value = 0)
    

    -- Returns
    -------
    Simulation results

    """

    try:
        #Check if inputs are list, and if not list them (i.e. if there is only one agent). To be deleted/adapted later depending on the input formats
        SE465 = list_check_input(SE465)
        A_0 = list_check_input(A_0)
        SE490 = list_check_input(SE490)
        SE420 = list_check_input(SE420)
        SE410 = list_check_input(SE410)
        agents_id = list_check_input(agents_id)
        land_ha = list_check_input(land_ha)

        nr_agents = len(agents_id)

        ##------------ PARAMETERS ------------##
        sim_period = sim_period + 1             # Simulation period
        i = sim_period*2               
        cte_a = 0.25                   # Constant for final constrain of A (fixed assets) variable
        shape = [sim_period, nr_agents]# Shape of the final matrixes
        a_factor = AversionRisk        # Vector of discount factors  --> TO BE ADDED AS INPUT OF THE ALGORITHM

        ## ---- Matrix to save results
        a_f = np.zeros(shape)          # Matrix to save final values of A for all agents
        lt_f = np.zeros(shape)         # Matrix to save final values of LT for all agents
        ca_f = np.zeros(shape)         # Matrix to save final values of CA for all agents
        fa_f = np.zeros(shape)         # Matrix to save final values of FA for all agents
        d_f = np.zeros(shape)          # Matrix to save final values of D for all agents
        lr_f = np.zeros(shape)         # Matrix to save final values of LR for all agents
        e_f = np.zeros(shape)          # Matrix to save final values of E for all agents
        sr_f = np.zeros(shape)         # Matrix to save final values of SR for all agents
        fni_f = np.zeros(shape)        # Matrix to save final values of FNI for all agents
        gfi_f = np.zeros(shape)        # Matrix to save final values of GFI for all agents

        a_ha_f = np.zeros(shape)       # Matrix to save final values of Land size (ha) for all agents
        a_va_f = np.zeros(shape)       # Matrix to save final values of Land value (€/ha) for all agents
        prod_ha_f = np.zeros(shape)    # Matrix to save final values of Land production (€/ha) for all agents
        u1_ha_f = np.zeros(shape)      # Matrix to save final values of land, in ha, for sell/buy

        u1_f = np.zeros(shape)  
        u2_f = np.zeros(shape)   
        np_f = np.zeros(shape)         # Matrix to save final values of NP for all agents
        year_f = np.zeros(shape)

        #######################
        # Declaring constants #
        #######################
        ##------------ CONSTANTS ------------##
        R = 0.005       #Interest
        MEPI = 5000      #Multiple Effects Public Income (MEPI) indicator - minimum annual living income established by the national government for year 
        ML = 30         #Average maturity of the loans
        EPS = 0.05       #SRobj
        COV = 0.15      #Farming overhead costs
        X = np.array([*range(0, i + 1)])
        C_ECON = 10     #Medium length of the economical cycle
        PYLD = 0.15 + 0.05 * np.sin(2 * np.pi * X / (C_ECON))   #Average ratio of value of production per value of land
        A_va_average = average_ha_va # €/ha

        ##------------ MODEL Iniciatilization ------------##
        model = None
        model = gp.Model("Financial Model")

        #######################
        # Declaring variables #
        #######################
        # HOW TO ADD variables: addVars(X_size, Y_size, vtype, upper_bond, bottom_bond, name, obj = by def 0, )

        # decision variables (first decisions initialised with zero value)
        u1 = model.addVars(i + 1, nr_agents, vtype = GRB.CONTINUOUS, lb=float("-inf"), name = "Decision variable 1 - Buy/sell land")
        # u1_aux = model.addVars(i + 1, nr_agents, vtype = GRB.CONTINUOUS, lb=float("-inf"), name = "Decision variable 1 - aux var to constraint min value")

        u2 = model.addVars(i + 1, nr_agents, vtype = GRB.CONTINUOUS, lb=float("-inf"), name = "Decision variable 2 - Loans")

        # Current assets
        ca = model.addVars(i + 1, nr_agents, vtype = GRB.CONTINUOUS, lb=float("-inf"), name = "Current Assets")

        # Fixed assets
        fa = model.addVars(i + 1, nr_agents, vtype = GRB.CONTINUOUS, lb=float("-inf"), name = "Fixed Assets")

        # Long term liabilities
        lt = model.addVars(i + 1, nr_agents, vtype = GRB.CONTINUOUS, lb=float("-inf"), name = "Long term liabilities")

        # Deposits
        d = model.addVars(i + 1, nr_agents, vtype = GRB.CONTINUOUS, lb=float("-inf"), name = "Deposits")

        # Auxiliary deposits variables to perform a "max" operation during optimization
        aux_d = model.addVars(i + 1, nr_agents, vtype = GRB.CONTINUOUS, lb=float("-inf"), name = "Aux Deposits")

        # Owned land
        a = model.addVars(i + 1, nr_agents, vtype = GRB.CONTINUOUS, lb=float("-inf"), name = "Owned Land")

        # Farm Net Income
        fni = model.addVars(i + 1, nr_agents, vtype = GRB.CONTINUOUS, lb=float("-inf"), name = "Farm Net Income")

        # Gross Farm Income
        gfi = model.addVars(i + 1, nr_agents, vtype = GRB.CONTINUOUS, lb=float("-inf"), name = "Gross Farm Income")

        # Equity
        e = model.addVars(i + 1, nr_agents, vtype = GRB.CONTINUOUS, lb=float("-inf"), name = "Equity")
        e_aux = np.zeros((i + 1, nr_agents))    

        # # Liquidity ratio
        lr = np.zeros((i + 1, nr_agents))

        # Solvency ratio
        sr = np.zeros((i + 1, nr_agents))

        netp = np.zeros((i + 1, nr_agents))

        # Land value and production
        A_ha = np.zeros((i + 1, nr_agents))     # Land size (ha)
        A_va = np.zeros((i + 1, nr_agents))     # Land ha value (€/ha)
        prod_ha = np.zeros((i + 1, nr_agents))  # Land production (€/ha)
        u1_ha = np.zeros((i + 1, nr_agents))    # Sell/buy land (ha)

        # Update land value and ha with transactions performed if any.
        for j in range(nr_agents): 
            u1[0, j] = 0
            A_ha[0, j] = land_ha[j]  # Land (ha)

            if LP_second_phase == True:
                u1[1, j] = 0 # Assume there are no transactions
                # Extract agent's transactions in the LMM (if any)
                # agent_transactions = LMM_transactions[(LMM_transactions['Buyer_FarmId'] == agents_id[j]) | (LMM_transactions['Seller_FarmId'] == agents_id[j])]
                agent_transactions = [transaction for transaction in LMM_transactions if transaction['Buyer_FarmId'] == agents_id[j] or transaction['Seller_FarmId'] == agents_id[j]]

                # If the agent got any transactions update u1 and land
                if len(agent_transactions) > 0:
                    if agent_transactions[0]['Buyer_FarmId'] == agents_id[j]:
                        buyer_role = True
                    else:
                        buyer_role = False
                    if buyer_role == True:
                        u1[1, j] = - agent_transactions[0]["SalePrice"]
                        A_ha[1, j] = land_ha[j] - agent_transactions[0]["Land_exchange"] 
                    elif buyer_role == False:
                        u1[1, j] = agent_transactions[0]["SalePrice"]
                        A_ha[1, j] = land_ha[j] + agent_transactions[0]["Land_exchange"]
        
            u2[0, j] = 0

            if SE465[j] == 0 and A_0[j] == 0:
                if SE420[j] != 0:
                    ca[0, j] = SE420[j]
                elif SE410[j] != 0:
                    ca[0, j] = SE410[j]
            else:
                ca[0, j] = SE465[j]

            d[0, j] = ca[0, j]
            fa[0, j] = A_0[j]  
            lt[0, j] = SE490[j]
            a[0, j] = A_0[j]    # Land (€)
            fni[0, j] = SE420[j]
            gfi[0, j] = SE410[j]
            e[0, j] = fa[0, j] + ca[0, j] - lt[0, j]
            e_aux[0, j] = e[0, j]
            lr[0, j] = ca[0, j] / (1 / ML) * lt[0, j]

            if lt[0, j] != 0:
                sr[0, j] = (lt[0, j]) / (fa[0, j] + ca[0, j] - lt[0, j])
            else:
                sr[0, j] = 0

            if A_ha[0, j] != 0:
                A_va[0, j] = A_0[j] / A_ha[0, j]
                prod_ha[0, j] = gfi[0, j] / A_ha[0, j]
            else:
                # The agent does not have land
                A_va[0, j] = A_va_average 
                prod_ha[0, j] = 0

        # Auxiliary variables to use "abs" operations in objective function
        aux_y1 = model.addVars(i + 1, nr_agents, vtype = GRB.CONTINUOUS, lb=float("-inf"), name = "Output aux")
        aux_y1_abs = model.addVars(i + 1, nr_agents, vtype = GRB.CONTINUOUS, lb=float("-inf"), name = "Output aux abs")

        ##------------ MODEL UPTADE ------------##
        model.update()
        model

        ########################
        # Optimization process #
        ########################

        # -- Simulate each year 
        for aux_j in range(1, sim_period):
            # y = np.zeros(nr_agents)   #Initialize objective function - It is set to 0 at the begining (of the simulation) on each year
            y = [0 for _ in range(nr_agents)]

            # -- Prediction window.
            for aux_i in range(aux_j, aux_j + sim_period + 1):     
                for j in range(nr_agents):
                
                    # ---- Equation ---- #
                    a[aux_i, j] = a[aux_i - 1, j] + u1[aux_i, j] 
                    fa[aux_i, j] = a[aux_i, j]                                          # Total fixed assets

                    lt[aux_i, j] = (1 - 1 / ML) * lt[aux_i - 1, j] + u2[aux_i, j]       # Long term liabilities
                    gfi[aux_i, j] = (1 - COV) * 2 * PYLD[aux_i] * a[aux_i, j]               # Gross Farm Income 
                    # gfi[aux_i, j] = a[aux_i, j] * (land_avg_prod - land_avg_costs) 

                    fni[aux_i, j] = (
                        gfi[aux_i, j]
                        + SBT_c * a[aux_i, j]
                        + SBT_nc
                        - VAT
                        - EFT
                        - 1 / ML * lt[aux_i - 1, j]
                        - u1[aux_i, j]
                    )

                    # ---- Inequation - Constraints ---- #
                    # Deposits             
                    model.addConstr(
                        d[aux_i, j] == (1 + R) * d[aux_i - 1, j] + fni[aux_i - 1, j] - NFM * MEPI + u2[aux_i, j],
                        name = "Deposits calculation"
                        )
                    # d[aux_i, j] = (1 + R) * d[aux_i - 1, j] + fni[aux_i - 1, j] - NFM * MEPI + u2[aux_i, j]

                    model.addConstr(d[aux_i, j] >= 0, name = "Deposits positivity")

                    # Currrent assets
                    # model.addConstr(ca[aux_i, j] == d[aux_i, j], name = "Current Assets equation")
                    ca[aux_i, j] = d[aux_i, j]
                    # Equity   
                    model.addConstr(
                        e[aux_i, j] == fa[aux_i, j] + ca[aux_i, j] - lt[aux_i, j],
                        name = "Equity equalty"
                    )
                                                                    
                    # TODO_ Check if it works without this constraint
                    # model.addConstr(
                    #     e[aux_i, j] >= 0.00001,
                    #     name = "Equity positivity"
                    # )

                    # Parameters limits                     
                    if LP_second_phase == True and aux_i == 1:
                        pass
                    else:
                        # model.addConstr(gfi[aux_i, j] >= 0)
                        model.addConstr(a[aux_i, j] >= 0)
                        
                    # Constrains for decision variables
                    # -- Loan adquisition
                    model.addConstr(u2[aux_i, j] >= 0)                       # Loans cannot be negative
                    model.addConstr(u2[aux_i, j] <= 10 * e[aux_i - 1, j])            # Loans adquisition limitation. Cannot be higher then 10*equity

                    # -- Land buy/sell expectations
                    if LP_second_phase == True and aux_i == 1:
                        pass
                    else:
                        model.addConstr(u1[aux_i, j] <= d[aux_i - 1, j])      # It is not possible to invest in land buy above the deposits capacity    
                        model.addConstr(u1[aux_i, j] >= -a[aux_i - 1, j])     # It is not possible to sell more land than available
                        model.addConstr(u1[aux_i, j] <= u2[aux_i, j]) 
                        # Sell or buy a minimum amount of land. It is not possible to sell/buy e.g. 0.1 ha
                        # min_land_transaction = A_va[0, j] / 2   # Transaction minimum half ha value
                        # model.addConstr(u1_aux[aux_i, j] == u1[aux_i, j] - min_land_transaction)
                        # model.addConstr(u1[aux_i, j] == max(0, u1_aux[aux_i, j] ))



                    # ---- Objective Function ---- #
                    alfa = a_factor ** (aux_i - aux_j)      #Discount factor - high a_factor (riskAversion) higer importance to the future
                    model.addConstr(
                        aux_y1[aux_i, j] == lt[aux_i, j] - EPS * e[aux_i, j]
                    )
                    model.addConstr(aux_y1_abs[aux_i, j] == gp.abs_(aux_y1[aux_i, j]))

                    beta = 0.8
                    # Obj. function
                    y[j] = y[j] + alfa * ( beta * (fni[aux_i, j] / e[aux_j - 1, j]) - (1 - beta) * aux_y1_abs[aux_i, j])
                    # y[j] = y[j] + alfa * ( (fni[aux_i, j] ) - aux_y1_abs[aux_i, j])

            model.Params.OutputFlag = 0  # Set to 0 to suppress output during optimization

            # Multi-agent optimisation check
            if nr_agents > 1:
                multi_objective_scalarization(model, y)
            else:
                # model.setObjective(y[0], GRB.MINIMIZE)
                model.setObjective(y[0], GRB.MAXIMIZE)

            model.update()
            model.optimize()

            # -- Check optimisation problem feasibility
            if model.Status == 4 or model.Status == 3 or model.Status == 5:
                print("Model INFEASIBLE. STOP optimisation")
                return "error"

            # Extracting and storing values in variable matrixes
            for j in range(nr_agents):
                if LP_second_phase == True and aux_j == 1:
                    # This is necessary because on the second run, some variables are pre-set and they don't act as variables
                    a[aux_j, j] = a[aux_j, j]
                    lt[aux_j, j] = lt[aux_j, j].getValue()
                    d[aux_j, j] = d[aux_j, j].X
                    ca[aux_j, j] = ca[aux_j, j].X
                    fa[aux_j, j] = fa[aux_j, j]
                    gfi[aux_j, j] = gfi[aux_j, j]
                    fni[aux_j, j] = fni[aux_j, j]
                    e[aux_j, j] = e[aux_j, j].X
                    e_aux[aux_j, j] = e[aux_j, j] 
                    lr[aux_j, j] = ca[aux_j, j] / ( 1 / ML) * lt[aux_j-1, j]
                    sr[aux_j, j] = (lt[aux_j, j]) / (
                        ca[aux_j, j] + fa[aux_j, j] - lt[aux_j, j]
                        )
                    u1[aux_j, j] = u1[aux_j, j]
                    u2[aux_j, j] = u2[aux_j, j].X
                
                else:
                    a[aux_j, j] = a[aux_j, j].getValue()
                    lt[aux_j, j] = lt[aux_j, j].getValue()
                    d[aux_j, j] = d[aux_j, j].X
                    ca[aux_j, j] = ca[aux_j, j].X
                    fa[aux_j, j] = fa[aux_j, j].getValue()
                    gfi[aux_j, j] = gfi[aux_j, j].getValue()
                    fni[aux_j, j] = fni[aux_j, j].getValue()
                    e[aux_j, j] = e[aux_j, j].X
                    e_aux[aux_j, j] = e[aux_j, j] 
                    lr[aux_j, j] = ca[aux_j, j] / ( 1 / ML) * lt[aux_j-1, j]
                    sr[aux_j, j] = (lt[aux_j, j]) / (
                        ca[aux_j, j] + fa[aux_j, j] - lt[aux_j, j]
                        )
                    u1[aux_j, j] = u1[aux_j, j].X
                    u2[aux_j, j] = u2[aux_j, j].X

                    # Calculate land production (€/ha)
                    if A_va[aux_j - 1, j] == 0:
                        A_va[aux_j - 1, j] = A_va_average

                    u1_ha[aux_j, j] = u1[aux_j, j] / A_va[aux_j - 1, j]
                    A_ha[aux_j, j] = A_ha[aux_j - 1, j] + u1_ha[aux_j, j]
                
                if A_ha[aux_j, j] != 0:
                    A_va[aux_j, j] = a[aux_j, j] / A_ha[aux_j, j]
                    prod_ha[aux_j, j] = gfi[aux_j, j] / A_ha[aux_j, j]
                else:
                    A_va[aux_j, j] = 0
                    prod_ha[aux_j, j] = 0

        # Storing the total results of the optimization for agent 
        for v in range(sim_period):
            for j in range(nr_agents):
                a_f[v, j] = a[v, j]
                lt_f[v, j] = lt[v, j]
                ca_f[v, j] = ca[v, j]
                fa_f[v, j] = fa[v, j]
                d_f[v, j] = d[v, j]
                lr_f[v, j] = lr[v, j]
                sr_f[v, j] = sr[v, j]
                fni_f[v, j] = fni[v, j]
                gfi_f[v, j] = gfi[v, j]
                e_f[v, j] = e[v, j]
                np_f[v, j] = netp[v, j]
                u1_f[v, j] = u1[v, j]
                u2_f[v, j] = u2[v, j]

                a_ha_f[v, j] = A_ha[v, j]
                a_va_f[v, j] = A_va[v, j]
                prod_ha_f[v, j] = prod_ha[v, j]
                u1_ha_f[v, j] = u1_ha[v, j]

            year_f[v, j] = year + v


        # Storing the total results of the optimization in a dataframe
        agents_LP_optimisation = pd.DataFrame()
        for j in range(nr_agents):
            agent_LP_opt = pd.DataFrame()
            agent_LP_opt["a_f"] = a_f[:, j]
            agent_LP_opt["lt_f"] = lt_f[:, j]
            agent_LP_opt["ca_f"] = ca_f[:, j]
            agent_LP_opt["fa_f"] = fa_f[:, j]
            agent_LP_opt["d_f"] = d_f[:, j]
            agent_LP_opt["lr_f"] = lr_f[:, j]
            agent_LP_opt["sr_f"] = sr_f[:, j]
            agent_LP_opt["fni_f"] = fni_f[:, j]
            agent_LP_opt["gfi_f"] = gfi_f[:, j]
            agent_LP_opt["e_f"] = e_f[:, j]
            agent_LP_opt["u1_f"] = u1_f[:, j]
            agent_LP_opt["u2_f"] = u2_f[:, j]

            agent_LP_opt["a_ha_f"] = a_ha_f[:, j]
            agent_LP_opt["a_va_f"] = a_va_f[:, j]
            agent_LP_opt["prod_ha_f"] = prod_ha_f[:, j]
            agent_LP_opt["u1_ha_f"] = u1_ha_f[:, j]

            agent_LP_opt["agent_id"] = agents_id[j]
            agent_LP_opt["Year"] = year_f[:, j]

            agents_LP_optimisation = pd.concat([agents_LP_optimisation, agent_LP_opt])

        return agents_LP_optimisation
    except Exception as e:
        print(e)
        return "error"

def buy_sell_match(buyers_c, sellers_c):
    min_u1_diff = np.inf

    for _, buyer in buyers_c.iterrows():
        for _, seller in sellers_c.iterrows():
            u1_diff = np.abs(buyer["u1_f"] + seller["u1_f"])     
            
            if u1_diff < min_u1_diff:
                min_u1_diff = u1_diff
                seller_id = int(seller["agent_id"])
                buyer_id = int(buyer["agent_id"])
    return seller_id, buyer_id

def LMM(agents_LP_decisions, productions_LMM):
    """
    Land Market Module (LMM) 

    Parameters:
    ----------
    Datasets containing below information for each agent.
        agent_id
        Year
        Year_id

        agents_LP_decisions:
        u1_f: decision variable from LP first stage optimisation - land to be sold/bought in €
        u1_ha_f: land to be sold/bought in ha

        agents_productions:
        ProductionId
        ha_value: agent ha value (€/ha)
        land_value (€)
        land_ha (ha)

    Returns:
    -------
    Matched transactions DataFrame
        ProductionId (long)
        Seller_FarmId (long) (origin Farm Id)
        Buyer_FarmId (long) (destination Farm Id)
        Percentage (float)
        Land_exchange
        SalePrice (float)
        YearId(long)
    """
    # Separate buyers and sellers
    buyers = agents_LP_decisions[agents_LP_decisions['u1_f'] > 0]
    sellers = agents_LP_decisions[agents_LP_decisions['u1_f'] < 0]    

    buyers = buyers[buyers["u1_ha_f"] > 0.5]
    sellers = sellers[sellers["u1_ha_f"] < -0.5]


    buyers_c = buyers.copy()
    sellers_c = sellers.copy()

    # Match buyers and sellers
    matched_transactions = []
    land_Transactions = []

    seller_productionId = []

    # Check if there are buyers and sellers
    if buyers.shape[0] != 0 and sellers.shape[0] != 0:
        while len(buyers_c) > 0 and len(sellers_c) > 0:
            seller_production_sold:list[tuple:(int, float)] = []

            # Match between sellers and buyers
            seller_id, buyer_id = buy_sell_match(buyers_c, sellers_c)
            
            # Extract agents
            buyer = buyers[buyers["agent_id"] == buyer_id]
            seller = sellers[sellers["agent_id"] == seller_id]

            buyer_productions = productions_LMM[productions_LMM["agent_id"] == buyer_id]
            seller_productions = productions_LMM[productions_LMM["agent_id"] == seller_id]

            buyer_offer = buyer['u1_f'].values[0]
            seller_offer = -seller['u1_f'].values[0]

            buyer_offer_ha = buyer['u1_ha_f'].values[0]
            seller_offer_ha = -seller['u1_ha_f'].values[0]
            # Perform transaction 
            # 1- Set the sale price and land exchanged
            if buyer_offer < seller_offer:
                transaction_price = buyer_offer
                if buyer_offer_ha < seller_offer_ha:
                    land_exchange = (buyer_offer * seller_offer_ha) / (seller_offer)
                else:
                    land_exchange = - seller['u1_ha_f'].values[0]
            else: 
                transaction_price = seller_offer
                land_exchange = seller_offer_ha

            # 2.1 - Check if the seller got several productions
            if len(seller_productions) > 1:
                land_exchange_c = land_exchange
                seller_productions_sorted = seller_productions.sort_values("land_ha", ascending = False)

                for _, seller_prod in seller_productions_sorted.iterrows():
                    # Production ha higher then the land to be exchanged --> Calculate the percentage of production exchanged
                    if seller_prod["land_ha"] >= land_exchange_c:
                        seller_production_sold.append((seller_prod["ProductionId"], land_exchange_c / seller_prod["land_ha"]))

                        # seller_productionId.append(seller_prod["ProductionId"])
                        # land_transfer_perc = (land_exchange / seller_prod["land_ha"])
                        break

                    # Production ha lower then the land to be exchanged --> percentage of land to be exchanged 1
                    else:
                        land_exchange_c = land_exchange_c - seller_prod["land_ha"]
                        seller_production_sold.append((seller_prod["ProductionId"], 1.0))
                        
                        # seller_productionId.append(seller_prod["ProductionId"])
                        # land_transfer_perc = 1.0
                        if land_exchange_c == 0:
                            break

            else:
                seller_production_sold.append((seller_productions["ProductionId"].values[0], land_exchange / seller_productions['land_ha'].values[0]))

                # seller_productionId.append(seller_productions["ProductionId"].values[0])
                # land_transfer_perc = (land_exchange / seller_productions['land_ha'].values[0])

            # Drop agent
            sellers_c = sellers_c[sellers_c['agent_id'] != seller_id]
            buyers_c = buyers_c[buyers_c['agent_id'] != buyer_id]
      
            for seller_prod_sold in seller_production_sold:
                prod_Id = seller_prod_sold[0]
                land_transfer_perc = seller_prod_sold[1]

                if land_transfer_perc > 1:
                    print("Error on land percentage transfer calculation")

                matched_transactions.append({
                    'productionId': prod_Id,
                    'yearId': seller_productions['year_id'].values[0],
                    'Buyer_FarmId': buyer_id,
                    'Seller_FarmId': seller_id,
                    'SalePrice': transaction_price,
                    'Percentage': land_transfer_perc,
                    'Land_exchange': land_exchange
                })     
                land_Transactions.append({
                    'productionId': prod_Id,
                    'yearId': seller_productions['year_id'].values[0],
                    'destinationFarmId': buyer_id,
                    'salePrice': transaction_price,
                    'percentage': land_transfer_perc,
                })     

    else: 
        print("No buyers or sellers")

    return matched_transactions, land_Transactions 


def holder_retire_ev(holder_age, successor_age):
    holder_retires = False

    if successor_age >= 18:
        if holder_age > 64:
            holder_retires = True

    return holder_retires


def gen_transactions_samples():
    # Sample data for buyers and sellers
    buyer1 = {'agent_id': 41544, 'Year': 2022}
    buyer2 = {'agent_id': 41545, 'Year': 2022}
    seller1 = {'agent_id': 41546, 'ProductionId': 'P1'}
    seller2 = {'agent_id': 41547, 'ProductionId': 'P2'}

    # Sample transactions
    transactions = [
        {'buyer': buyer1, 'seller': seller1, 'transaction_price': 10000, 'land_exchange_perc': 0.2, 'land_exchange': 10},
        {'buyer': buyer2, 'seller': seller2, 'transaction_price': 15000, 'land_exchange_perc': 0.3, 'land_exchange': 15},
    ]

    # Initialize the list for matched transactions
    matched_transactions = []

    # Append transactions to the list
    for transaction in transactions:
        buyer = transaction['buyer']
        seller = transaction['seller']
        transaction_price = transaction['transaction_price']
        land_exchange_perc = transaction['land_exchange_perc']
        land_exchange = transaction['land_exchange']

        matched_transactions.append({
            'ProductionId': seller['ProductionId'],
            'YearID': buyer['Year'],
            'Buyer_FarmId': buyer['agent_id'],
            'Seller_FarmId': seller['agent_id'],
            'SalePrice': transaction_price,
            'Percentage': land_exchange_perc,
            'Land_exchange': land_exchange
        })
    return matched_transactions

def plot_lp_results(LP_results):
    """
    Function to plot long period optimisation results

    Parameters - inputs
    ----------
    LP_results = dataframe with long period optimisation results
       
    Returns
    -------
    Plots

    """
    
    plt.figure()
    plt.subplot(321)
    plt.plot(LP_results.a_f)
    plt.title("Owned Land")
    plt.subplot(322)
    plt.plot(LP_results.lt_f)
    plt.title("Long term liabilities")
    plt.subplot(323)
    plt.plot(LP_results.ca_f)
    plt.title("Current Assets")
    plt.subplot(324)
    plt.plot(LP_results.fa_f)
    plt.title("Fixed Assets")
    plt.subplots_adjust(left=0.1,
            bottom=0.1, 
            right=0.9, 
            top=0.9, 
            wspace=0.4, 
            hspace=0.4)
    
    plt.figure()
    plt.subplot(321)
    plt.plot(LP_results.d_f)
    plt.title("Deposits")
    plt.subplot(322)
    plt.plot(LP_results.lr_f)
    plt.title("Liquidity ratio")
    plt.subplot(323)
    plt.plot(LP_results.sr_f)
    plt.title("Solvency ratio")
    plt.subplot(324)
    plt.plot(LP_results.fni_f)
    plt.title("Farm Net Income")
    plt.subplot(325)
    plt.plot(LP_results.gfi_f)
    plt.title("Gross Farm Income")
    plt.subplot(326)
    plt.plot(LP_results.e_f)
    plt.title("Equity")
    plt.subplots_adjust(left=0.1,
            bottom=0.1, 
            right=0.9, 
            top=0.9, 
            wspace=0.4, 
            hspace=0.4)
    
    LP_results[["u0_f", "u1_f"]].plot()
    plt.title("Decision actions (u0 = sell-/buy+ land | u1 = loans)")
    plt.xlabel("Years")
    plt.ylabel("Value (€)")
    plt.show()
